<a href="https://colab.research.google.com/github/PavelNovikov888/SciPy/blob/master/SciPy_%D0%A3%D1%81%D1%82%D0%B0%D0%BD%D0%BE%D0%B2%D0%BA%D0%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [86]:
# !pip install scipy

После установки давайте проверим, что всё работает, и заодно познакомимся с одной важной спецификой библиотеки.
Импортируем пакет с оптимизаторами.

In [87]:
from scipy import optimize
# Для просмотра документации импортируем numpy
import numpy as np
# Посмотрим документацию минимизатора функции
np.info(optimize.fmin)
from scipy import sparse
from numpy.random import rand

 fmin(func, x0, args=(), xtol=0.0001, ftol=0.0001, maxiter=None, maxfun=None,
      full_output=0, disp=1, retall=0, callback=None, initial_simplex=None)

Minimize a function using the downhill simplex algorithm.

This algorithm only uses function values, not derivatives or second
derivatives.

Parameters
----------
func : callable func(x,*args)
    The objective function to be minimized.
x0 : ndarray
    Initial guess.
args : tuple, optional
    Extra arguments passed to func, i.e., ``f(x,*args)``.
xtol : float, optional
    Absolute error in xopt between iterations that is acceptable for
    convergence.
ftol : number, optional
    Absolute error in func(xopt) between iterations that is acceptable for
    convergence.
maxiter : int, optional
    Maximum number of iterations to perform.
maxfun : number, optional
    Maximum number of function evaluations to make.
full_output : bool, optional
    Set to True if fopt and warnflag outputs are desired.
disp : bool, optional
    Set to Tru

1. Научимся создавать матрицы разных типов:

In [88]:
# from scipy import sparse
# from numpy.random import rand

In [89]:
sample_csr = sparse.csr_matrix([[1, 2, 0], [0, 0, 3], [4, 0, 5]])

In [90]:
print(sample_csr)

  (0, 0)	1
  (0, 1)	2
  (1, 2)	3
  (2, 0)	4
  (2, 2)	5


In [91]:
# CSС-матрица
row = np.array([0, 2, 2, 0, 1, 2])
col = np.array([0, 0, 1, 2, 2, 2])
data = np.array([1, 2, 3, 4, 5, 6])

sample_csc = sparse.csc_matrix((data, (row, col)), shape=(3, 3))

In [92]:
print(sample_csc)

  (0, 0)	1
  (2, 0)	2
  (2, 1)	3
  (0, 2)	4
  (1, 2)	5
  (2, 2)	6


In [93]:
# COO-матрица
I = np.array([0,3,1,0])
J = np.array([0,3,1,2])
V = np.array([4,5,7,9])

sample_coo = sparse.coo_matrix((V,(I,J)),shape=(4,4))

In [94]:
print(sample_coo)

  (0, 0)	4
  (3, 3)	5
  (1, 1)	7
  (0, 2)	9


In [95]:
# LIL матрица
sample_lil = sparse.lil_matrix((0, 101))

In [96]:
print(sample_lil)

In [97]:
#  DOK-матрица
sample_dok = sparse.dok_matrix((5, 5), dtype=np.float32)

In [98]:
print(sample_dok)

2. Научимся заполнять матрицы. К счастью, большая часть механики уже реализована за нас:

In [99]:
d = np.ones(10)
print(d)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]


In [100]:
# from numpy.random import rand

# LIL matrix
sample_lil = sparse.lil_matrix((1000,1000))
sample_lil[1, :100] = np.ones(100)
sample_lil[1, 100:200] = sample_lil[0, :100]
sample_lil.setdiag(rand(1000))

for i in range(5):
    for j in range(5):
        sample_dok [i, j] = i + j

К сожалению, COO-, CSC-, CSR-матрицы очень плохо изменяемы после создания — большая часть операций для них не определена. Поэтому COO-матрицы будем считать неизменяемыми. Для обновления CSC и CSR используется матричное сложение или перевод в dense-формат с последующим обновлением и переводом обратно в sparse-формат. Давайте посмотрим, как это делается, на примере CSR:


In [101]:
 # Переведем в dense и исправим необходимые значения
tmp_filled = sample_csr.toarray()
tmp_filled[0, 0] = 10
sample_csr_new = sparse.csr_matrix(tmp_filled)

#Аналогично через сложение двух матриц
tmp_csr = sparse.csr_matrix([[5, 0, 0], [0, 0, 0], [0, 0, 0]])
sample_csr_new = sample_csr + tmp_csr

3. С матрицами можно делать большинство матричных и матрично-векторных операций, которые мы совершали с помощью numpy.
## Одно важное уточнение: матричные операции мы можем совершать только между матрицами одного формата.

In [102]:
v = np.array([1, 0, -1])
tmp_csr = sparse.csr_matrix([[5, 0, 0], [0, 0, 0], [0, 0, 0]])
sample_csr = sample_csr + tmp_csr
sample_csr = sample_csr.dot(v)

In [103]:
v = np.array([1, 0, -1])
tmp_csc = sparse.csc_matrix([[5, 0, 0], [0, 0, 0], [0, 0, 0]])
sample_csc = sample_csc + tmp_csc
sample_csc = sample_csc.dot(v)

In [109]:
sample_lil_tmp = sparse.lil_matrix((1000, 1000))
sample_lil1 = sample_lil + sample_lil_tmp

In [111]:
sample_lil = sparse.lil_matrix((3, 3))
v = np.array([1, 0, -1])
sample_lil_tmp = sparse.lil_matrix((3, 3))
sample_lil = sample_lil.dot(v) + sample_lil_tmp
sample_lil

matrix([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

Создадим матрицу CSR размером 3*4, в которой первый столбец заполнен единицами, а третий — четвёрками.   
Создайте sparse CSR-матрицу с помощью трёх массивов и запишите в переменную tmp.   
После каждой запятой обязательно добавьте пробел.

In [124]:
row = np.array([0, 0, 1, 1, 2, 2])
col = np.array([0, 2, 0, 2, 0, 2])
data = np.array([1, 4, 1, 4, 1, 4])
tmp = sparse.csr_matrix((data, (row, col)), shape=(3, 4)).toarray()
tmp

array([[1, 0, 4, 0],
       [1, 0, 4, 0],
       [1, 0, 4, 0]])

In [125]:
tmp = sparse.csr_matrix(([1, 4, 1, 4, 1, 4], ([0, 0, 1, 1, 2, 2], [0, 2, 0, 2, 0, 2])), shape=(3, 4)).toarray()
tmp

array([[1, 0, 4, 0],
       [1, 0, 4, 0],
       [1, 0, 4, 0]])